In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',None)
from sklearn.linear_model import LinearRegression,LogisticRegression
import seaborn as sns


/Users/sgcx198/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/sgcx198/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/sgcx198/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [ ]:
path = '../data/'
data = pd.read_csv(path+'creditcard.csv')
data.head()

In [ ]:
data.info()

In [ ]:
# 数据确实情况
data.isnull().sum()

In [ ]:
# 数据相关性
corr = data.corr()
plt.figure(figsize=(50.40))
sns.heatmap(corr,annot=True,vmax=1,xticklabels=True,yticklabel=True,square=True,cmap="YlGnBu")
plt.title('ss')
plt.show()

In [ ]:
# 查看类别
data.['Class'].value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
data['Amount'] = StandardScaler().fit_transform(np.array(data['Amount']).reshape(-1,1))

In [ ]:
# 类别不平衡
# 下采样
undersample_seed = 1
num_Card_Fraud = data[data['Class'] == 1].shape[0]
num_Card_Fraud

In [ ]:
Normal_users = data[data['Class'] == 0].index.values
Normal_users_selection = np.random.RandomState(undersample_seed).choice(
    Normal_users,size=num_Card_Fraud,replace=False
)
len(Normal_users_selection)

In [ ]:
Card_Fraud = data[data['Class'] == 1].index.values
selected_users = list(set(Normal_users_selection).union(set(Card_Fraud)))

In [ ]:
new_train = data.loc[selected_users].copy()
new_train["Class"].value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler
new_train['Amount'] = StandardScaler().fit_transform(np.array(new_train['Amount']).reshape(-1,1))
feats = [i for i in new_train.columns if i not in ['Time','Class']]
train = new_train[feats]
label = new_train['Class']

In [ ]:
label.value_counts()

In [ ]:
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import GridSearchCV
lg = LogisticRegression()
clf = GridSearchCV(lg,param_grid=[{'C':np.arange(0.001,0.05,0.01)}],cv=5)
clf.fit(train,label)
best_model = clf.best_estimator_
clf.best_params_


In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(data[feats],data['Class'],test_size=0.3,random_state=42)
print("or_train: ",len(train_x))
print("or_test: ",len(test_x))
print("or_data: ",len(data))
# 切分下采样训练集和下采样测试集
train_x_us, test_x_us, train_y_us, test_y_us = train_test_split(new_train[feats],new_train["Class"],test_size = 0.3,random_state = 42)
print('-----------------------------------------------------------')
print("undersample_len_of_train: ", len(train_x_us))
print("undersample_len_of_test: ", len(test_x_us))
print("undersample_len_of_train: ", len(new_train))

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import confusion_matrix,recall_score,classification_report


In [ ]:
import matplotlib.pyplot as plt

def plt_confusion(cm,classes,titile='Confusion matrix',cmap=plt.cm.Blues):
    plt.imshow(cm,interpolation='nearest',cmap=cmap)
    plt.title(titile)
    plt.colorbar()
    set_of_class = np.arange(len(classes))
    plt.xticks(set_of_class,classes,rotation=0)
    